Ahmed Ali 22i-0825 E

Q1 Local Beam Search

In [40]:


def initial_state(graph, pre_assigned):
    state = []
    for key in graph.keys():
        if key in pre_assigned:
            state.append(pre_assigned[key])
        else:
            state.append(None)
    return state

def check_one_hop(graph, state, i):
    neighbours = graph[i]
    for neighbour in neighbours:
        if state[neighbour] == state[i]:
            return False
    return True

def get_vertices_per_color(state,colors):
    vertices_per_color = {}
    for color in colors:
        vertices_per_color[color] = 0
        for i in range(len(state)):
            if state[i] == color:
                vertices_per_color[color] += 1
    return vertices_per_color
    
def get_two_hop_conflicts(graph, state):
    conflicts = 0
    for i in range(len(state)):
        neighbours = graph[i]
        two_hop_neighbours = []
        for n in neighbours:
            two_hop = graph[n]
            for l in two_hop:
                two_hop_neighbours.append(l)
        two_hop_neighbours = list(set(two_hop_neighbours))
        two_hop_neighbours.remove(i)    # Remove the vertex itself
        for neighbour in two_hop_neighbours:
            if state[neighbour] == state[i]:
                conflicts += 1

                
    return conflicts

def get_degree(graph, vertex):
    return len(graph[vertex])

def cube(x):
    return x**3


# Current vertex is the vertex we just colored
def get_heuristic(graph, state,colors,current_vertex):
    # Minimum color constraint
    filtered_state = [s for s in state if s is not None]
    h_color = 8 * cube(max(set(filtered_state))) if filtered_state else 0  # Number of unique colors used

    # Balance colors constraint
    w2 = 2
    vertices_per_color = get_vertices_per_color(state,colors)
    k = len(colors) # Number of colors
    ideal = len(state) / k
    #Find variance
    h_balance = w2 * (sum([(vertices_per_color[color] - ideal)**2 for color in colors]) / k)

    # Two hop constraint
    w3 = 3
    h_two_hop = w3 * get_two_hop_conflicts(graph, state)

    # High degree vertices constraint
    w4 = 2
    degree = get_degree(graph, current_vertex)
    h_degree = w4 * degree  # We will prefer coloring vertices with higher degree

    return h_color + h_balance + h_two_hop + h_degree

def find_successors(graph, state,colors):
    successors = []
    for i in range(len(state)):
        if state[i] is None:
            for color in colors:
                state[i] = color
                if check_one_hop(graph, state, i): # Check if the coloring is valid
                    successors.append((state.copy(),get_heuristic(graph,state.copy(),colors,i)))
            state[i] = None

    return successors


def beam_search(graph, initial_state, k,colors):
    frontier = []
    k_best = []
    found = False
    frontier = find_successors(graph, initial_state, colors)
   
    
    while not found:
        sorted_frontier = sorted(frontier, key=lambda x: x[1])
        sorted_frontier = sorted_frontier[:k]
        print("frontier ",frontier)
        frontier = []
        for state in sorted_frontier:
            if None not in state[0]:
                found = True
                k_best = sorted_frontier.copy()
                break
            else:
                frontier = frontier + find_successors(graph, state[0], colors)
                if len(frontier) == 0:
                    print("No solution found")
                    return None
    
    
    return k_best


def main():
    graph = {
        0: [1, 2],
        1: [0, 2, 3],
        2: [0, 1, 4],
        3: [1, 4],
        4: [2, 3, 5],
        5: [4]
    }


    pre_assigned = {
        0: 0,  # Vertex 0 is color 0 (e.g., red)
        5: 1   # Vertex 5 is color 1 (e.g., blue)
    }

    colors = [0,1,2,3]

    initial = initial_state(graph, pre_assigned)
    # print(initial)
    # print(find_successors(graph, initial, colors))
    ans = beam_search(graph, initial, 2, colors)
    print(ans)
    print("Number of colors used in optimal solution: ", (max(ans[0][0])) + 1)


if __name__ == '__main__':
    main()

frontier  [([0, 1, None, None, None, 1], 22.5), ([0, 2, None, None, None, 1], 77.5), ([0, 3, None, None, None, 1], 229.5), ([0, None, 1, None, None, 1], 28.5), ([0, None, 2, None, None, 1], 77.5), ([0, None, 3, None, None, 1], 229.5), ([0, None, None, 0, None, 1], 32.5), ([0, None, None, 1, None, 1], 32.5), ([0, None, None, 2, None, 1], 81.5), ([0, None, None, 3, None, 1], 233.5), ([0, None, None, None, 0, 1], 34.5), ([0, None, None, None, 2, 1], 83.5), ([0, None, None, None, 3, 1], 235.5)]
frontier  [([0, 1, 2, None, None, 1], 71.5), ([0, 1, 3, None, None, 1], 223.5), ([0, 1, None, 0, None, 1], 20.5), ([0, 1, None, 2, None, 1], 69.5), ([0, 1, None, 3, None, 1], 221.5), ([0, 1, None, None, 0, 1], 28.5), ([0, 1, None, None, 2, 1], 77.5), ([0, 1, None, None, 3, 1], 229.5), ([0, 2, 1, None, None, 1], 77.5), ([0, 3, 1, None, None, 1], 229.5), ([0, None, 1, 0, None, 1], 32.5), ([0, None, 1, 1, None, 1], 39.5), ([0, None, 1, 2, None, 1], 81.5), ([0, None, 1, 3, None, 1], 233.5), ([0, None, 1